## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\JoyceYu\Anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
input_shape = (32, 32, 3)
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Conv2D(32, (3,3) ,input_shape=(32,32,3),activation='relu'))
###classifier.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, (3,3),activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
#output_dim=100,activation=relu
classifier.add(Dense(100,activation='relu'))

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=512,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 5s 106us/step - loss: 1.4198 - acc: 0.5184
Epoch 2/10
50000/50000 [==============================] - 3s 63us/step - loss: 0.8722 - acc: 0.6964: 0s - loss: 0.8734 -
Epoch 3/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.6314 - acc: 0.7834: 1s
Epoch 4/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.4562 - acc: 0.8491
Epoch 5/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.3168 - acc: 0.9017: 1s 
Epoch 6/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.2011 - acc: 0.9438
Epoch 7/10
50000/50000 [==============================] - 3s 63us/step - loss: 0.1181 - acc: 0.9733
Epoch 8/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.0659 - acc: 0.9899
Epoch 9/10
50000/50000 [==============================] - 3s 64us/step - loss: 0.0357 - acc: 0.9971
Epoch 10/10
50000/50000 [==============================] - 3s 63us/st

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.8939565e-02, 5.9727058e-06, 3.7888640e-01, 1.9496883e-01,
        3.8710836e-01, 1.5832020e-05, 2.1622847e-05, 1.0951449e-05,
        4.2448846e-05, 2.2155101e-08]], dtype=float32)